In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tabulate import tabulate
import os
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
%matplotlib inline
import copy
import datetime
import lightgbm as lgb
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
import os
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, classification_report, confusion_matrix
import json
import ast
import time
from sklearn import linear_model

import warnings
warnings.filterwarnings('ignore')

import os
import glob

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.metrics import roc_curve


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
       

# Any results you write to the current directory are saved as output.

In [ ]:
def print_df(df):
    print(tabulate(df, headers="keys",tablefmt="html"))
    
RegularSeasonCompactResults = pd.read_csv('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
#print("size = ", RegularSeasonCompactResults.shape)
#print_df(RegularSeasonCompactResults.head())



data_dict = {}
data_dict1 = {}
for i in glob.glob('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/*'):
    name = i.split('/')[-1].split('.')[0]
    if name != 'MTeamSpellings':
        data_dict[name] = pd.read_csv(i)
    else:
        data_dict[name] = pd.read_csv(i, encoding='cp1252')

sub = pd.read_csv('../input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv')
#sub.head()

#data_dict.keys()
data_dict1 = data_dict.copy()

for name, df in data_dict.items():
    print(f'{name}: {df.shape}')





In [ ]:
# display teams list
data_dict['MTeams'].head()

In [ ]:
#Various names for each teams
data_dict['MTeamSpellings'].head()

In [ ]:
data_dict['MNCAATourneySeeds'].head()

In [ ]:
#Convert Alpha numeric seed to nummerical for comparision 
data_dict['MNCAATourneySeeds']['Seed'] = data_dict['MNCAATourneySeeds']['Seed'].apply(lambda x: int(x[1:3]))
data_dict['MNCAATourneySeeds'] = data_dict['MNCAATourneySeeds'][['Season', 'TeamID', 'Seed']]
data_dict['MNCAATourneySeeds'].head()

In [ ]:
x_temp =data_dict['MRegularSeasonCompactResults'].groupby('Season')[['WScore','LScore']].mean()

fig = plt.gcf()
fig.set_size_inches(28, 12)
plt.plot(x_temp.index,x_temp['WScore'],marker='o', markerfacecolor='green', markersize=12, color='green', linewidth=4)
plt.plot(x_temp.index,x_temp['LScore'],marker=7, markerfacecolor='red', markersize=12, color='red', linewidth=4)
plt.legend()

In [ ]:

df_temp = pd.merge(data_dict['MNCAATourneyCompactResults'], data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
df_temp = pd.merge(df_temp, data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
team_win_score = data_dict['MRegularSeasonCompactResults'].groupby(['Season', 'WTeamID'])['WScore'].mean().reset_index()
#print(team_win_score)
team_loss_score = data_dict['MRegularSeasonCompactResults'].groupby(['Season', 'LTeamID'])['LScore'].mean().reset_index()
df_temp = pd.merge(df_temp, team_win_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])
df_temp = pd.merge(df_temp, team_loss_score, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])
df_temp = pd.merge(df_temp, team_loss_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'])
df_temp = pd.merge(df_temp, team_win_score, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
df_temp.head()
#data_dict1['MRegularSeasonCompactResults'].head()
#df

df_temp['Score_Diff'] = df_temp['WScore_x'] - df_temp['WScore_y']
plt.style.use('fivethirtyeight')
df_temp['Score_Diff'] \
    .plot(kind='hist',
          bins=90,
          figsize=(15, 5),
          label='Mens',
          alpha=0.5)
plt.title('Score Differential')
plt.xlim(0,60)
plt.legend()
plt.show()

data_dict['MNCAATourneyCompactResults'].head()

Mens regular wining results

In [ ]:
data_dict['MRegularSeasonCompactResults']
data_dict['MRegularSeasonCompactResults'].groupby(['Season'])['WScore'].mean().plot();
plt.title('Mean scores of winning teams by season in regular plays');

In [ ]:

data_dict['MNCAATourneyCompactResults'].groupby(['Season'])['WScore'].mean().plot(kind='line');
plt.title('Mean scores of winning teams by season in tourneys');

In [ ]:
data_dict['MNCAATourneyCompactResults'].head()

In [ ]:
df = pd.merge(data_dict['MNCAATourneyCompactResults'], data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
df = pd.merge(df, data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
df = df.drop(['TeamID_x', 'TeamID_y', 'DayNum','NumOT','WLoc','WScore','LScore'], axis=1)
df['seed_diff'] = df['Seed_x'] - df['Seed_y']
df.head()

In [ ]:
data_dict['MNCAATourneyDetailedResults'].head()

In [ ]:
df_d = pd.merge(data_dict['MNCAATourneyCompactResults'], data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
df_d = pd.merge(df_d, data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
df_d = df_d.drop(['TeamID_x', 'TeamID_y', 'DayNum','NumOT','WLoc','WScore','LScore'], axis=1)
df_d['seed_diff'] = df_d['Seed_x'] - df_d['Seed_y']
cols = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']
# df_d['FGM_diff'] = df_d['WFGM'] - df_d['LFGM']
# df_d['FGA_diff'] = df_d['WFGA'] - df_d['LFGA']
# df_d['FGM3_diff'] = df_d['WFGM3'] - df_d['LFGM3']
# df_d['FGA3_diff'] = df_d['WFGA3'] - df_d['LFGA3']
# df_d['FTM_diff'] = df_d['WFTM'] - df_d['LFTM']
# df_d['FTA_diff'] = df_d['WFTA'] - df_d['LFTA']
# df_d['OR_diff'] = df_d['WOR'] - df_d['LOR']
# df_d['DR_diff'] = df_d['WDR'] - df_d['LDR']
# df_d['Ast_diff'] = df_d['WAst'] - df_d['LAst']
# df_d['TO_diff'] = df_d['WTO'] - df_d['LTO']
# df_d['Stl_diff'] = df_d['WStl'] - df_d['LStl']
# df_d['Blk_diff'] = df_d['WBlk'] - df_d['LBlk']
# df_d['PF_diff'] = df_d['WPF'] - df_d['LPF']
#df_d = df_d.drop(['WFGM', 'WFGA','WFGM3','WFGA3','WFTM','WFTA','WOR','WDR','WAst','WTO','WStl','WBlk','WPF','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA', 'LOR','LDR','LAst','LTO','LStl','LBlk','LPF'],axis=1)

df_d.head()
data_dict['MRegularSeasonCompactResults']['Season'] += 1
data_dict['MRegularSeasonDetailedResults']['Season'] += 1

team_win_score = data_dict['MRegularSeasonCompactResults'].groupby(['Season', 'WTeamID'])['WScore'].mean().reset_index()
#print(team_win_score)
team_loss_score = data_dict['MRegularSeasonCompactResults'].groupby(['Season', 'LTeamID'])['LScore'].mean().reset_index()
#team_loss_score
df_d = df_d.loc[(df_d['Season'] > 2004) & (df_d['Season'] < 2014)]
df_d = pd.merge(df_d, team_win_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])
df_d = pd.merge(df_d, team_loss_score, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])
df_d = pd.merge(df_d, team_loss_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'])
df_d = pd.merge(df_d, team_win_score, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
df_d.drop(['LTeamID_y', 'WTeamID_y'], axis=1, inplace=True)


team_win_FGM = data_dict['MRegularSeasonDetailedResults'].groupby(['Season', 'WTeamID'])['WFGM'].mean().reset_index()
#print(team_win_score)
team_loss_FGM = data_dict['MRegularSeasonDetailedResults'].groupby(['Season', 'LTeamID'])['LFGM'].mean().reset_index()
#print(team_loss_score)
df_d = pd.merge(df_d, team_win_FGM, how='left', left_on=['Season', 'WTeamID_x'], right_on=['Season', 'WTeamID'])
df_d = pd.merge(df_d, team_loss_FGM, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'LTeamID'])
df_d.drop(['LTeamID', 'WTeamID'], axis=1, inplace=True)
df_d = pd.merge(df_d, team_loss_FGM, how='left', left_on=['Season', 'WTeamID_x'], right_on=['Season', 'LTeamID'])
df_d = pd.merge(df_d, team_win_FGM, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
df_d.drop(['LTeamID', 'WTeamID'], axis=1, inplace=True)



#df_d = df_d.loc[(df['Season'] > 2003) & (df['Season'] < 2014)]
df_d.head()

In [ ]:
# data_dict['MRegularSeasonCompactResults']['Season'] += 1
# data_dict['MRegularSeasonDetailedResults']['Season'] += 1

In [ ]:

# team_win_score = data_dict['MRegularSeasonCompactResults'].groupby(['Season', 'WTeamID'])['WScore'].mean().reset_index()
# #print(team_win_score)
# team_loss_score = data_dict['MRegularSeasonCompactResults'].groupby(['Season', 'LTeamID'])['LScore'].mean().reset_index()
# #team_loss_score
# df_d = df_d.loc[(df_d['Season'] > 2003) & (df_d['Season'] < 2014)]
# df_d = pd.merge(df_d, team_win_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])
# df_d = pd.merge(df_d, team_loss_score, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])
# df_d = pd.merge(df_d, team_loss_score, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'])
# df_d = pd.merge(df_d, team_win_score, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
# df_d.drop(['LTeamID_y', 'WTeamID_y'], axis=1, inplace=True)


# team_win_FGM = data_dict['MRegularSeasonDetailedResults'].groupby(['Season', 'WTeamID'])['WFGM'].mean().reset_index()
# #print(team_win_score)
# team_loss_FGM = data_dict['MRegularSeasonDetailedResults'].groupby(['Season', 'LTeamID'])['LFGM'].mean().reset_index()
# #print(team_loss_score)
# df_d = pd.merge(df_d, team_win_FGM, how='left', left_on=['Season', 'WTeamID_x'], right_on=['Season', 'WTeamID'])
# df_d = pd.merge(df_d, team_loss_FGM, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'LTeamID'])
# df_d.drop(['LTeamID', 'WTeamID'], axis=1, inplace=True)
# df_d = pd.merge(df_d, team_loss_FGM, how='left', left_on=['Season', 'WTeamID_x'], right_on=['Season', 'LTeamID'])
# df_d = pd.merge(df_d, team_win_FGM, how='left', left_on=['Season', 'LTeamID_x'], right_on=['Season', 'WTeamID'])
# df_d.drop(['LTeamID', 'WTeamID'], axis=1, inplace=True)



# #df_d = df_d.loc[(df['Season'] > 2003) & (df['Season'] < 2014)]
# df_d.head()

In [ ]:
data_dict['MRegularSeasonDetailedResults'].head()

In [ ]:
data_dict['MRegularSeasonDetailedResults']['Season_join'] = data_dict['MRegularSeasonDetailedResults']['Season'] + 1

In [ ]:
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WFGA'].mean().plot();
plt.title('Average no of field goals Attempted vs Made by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WFGM'].mean().plot();

In [ ]:
plt.title('Average no of three pointers made by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WFGM3'].mean().plot();

In [ ]:
plt.title('Average no of three pointers attempted by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WFGA3'].mean().plot();
plt.title('Average no of three pointers attempted vs made by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WFGM3'].mean().plot();

In [ ]:
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WFTA'].mean().plot();
plt.title('Average no of free throws attempted and made(green) by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WFTM'].mean().plot();

In [ ]:
plt.title('Average No of offensive rebounds pulled by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WOR'].mean().plot();

In [ ]:
plt.title('Average No of defensive rebounds pulled by winning teams pwe season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WDR'].mean().plot();

In [ ]:
plt.title('Average No of assists by winning teams by season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WAst'].mean().plot();

In [ ]:
plt.title('Average No of turnovers committed by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WTO'].mean().plot();

In [ ]:
plt.title('Average No of steals accomplished by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WStl'].mean().plot();

In [ ]:
plt.title('Average No of blocks accomplished by winning teams per season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WBlk'].mean().plot();

In [ ]:
plt.title('Mean number of personal fouls committed by winning teams by season');
data_dict['MRegularSeasonDetailedResults'].groupby(['Season'])['WPF'].mean().plot();

In [ ]:
df.head()

In [ ]:
loss_df = df_d[df_d['WTeamID_x'] > df_d['LTeamID_x']]
win_df = df_d[df_d['WTeamID_x'] < df_d['LTeamID_x']]
win_df['target'] = 1
#win_df.columns = ['Season', 'Team1', 'Team2', 'Seed_1', 'Seed_2', 'seed_diff','FGM_diff','FGA_diff','FGM3_diff','FGA3_diff','FTM_diff','FTA_diff','OR_diff','DR_diff','Ast_diff','TO_diff','Stl_diff','Blk_diff','PF_diff','WScore_1', 'LScore_1', 'LScore_2', 'WScore_2', 'target']
win_df.columns = ['Season', 'Team1', 'Team2', 'Seed_1', 'Seed_2', 'seed_diff','WScore_1', 'LScore_1', 'LScore_2', 'WScore_2', 'WFGM_1','LFGM_1','LFGM_2','WFGM_2','target']
loss_df['target'] = 0
loss_df = loss_df[['Season', 'LTeamID_x', 'WTeamID_x', 'Seed_y', 'Seed_x', 'seed_diff','LScore_y', 'WScore_y', 'WScore_x', 'LScore_x', 'WFGM_x','LFGM_x','LFGM_y','WFGM_y','target']]
loss_df.columns = ['Season', 'Team1', 'Team2', 'Seed_1', 'Seed_2', 'seed_diff','WScore_1', 'LScore_1', 'LScore_2', 'WScore_2','WFGM_1','LFGM_1','LFGM_2','WFGM_2', 'target']
loss_df['seed_diff'] = -1 * loss_df['seed_diff']
new_df = win_df.append(loss_df)
new_df.head()

In [ ]:
sub = pd.read_csv('../input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv')
sub.head()
test = sub.copy()
sub['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
sub['Team1'] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub['Team2'] = sub['ID'].apply(lambda x: int(x.split('_')[2]))
#sub = pd.merge(sub, data_dict['MRegularSeasonDetailedResults'], how='left', left_on=['Season', 'Team1', 'Team2'], right_on=['Season', 'WTeamID','LTeamID'])
sub = pd.merge(sub, data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'Team1'], right_on=['Season', 'TeamID'])
sub = pd.merge(sub, data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'Team2'], right_on=['Season', 'TeamID'])
sub = pd.merge(sub, team_win_FGM, how='left', left_on=['Season', 'Team1'], right_on=['Season', 'WTeamID'])
sub = pd.merge(sub, team_loss_FGM, how='left', left_on=['Season', 'Team2'], right_on=['Season', 'LTeamID'])
sub.drop(['TeamID_x', 'TeamID_y','LTeamID','WTeamID'], axis=1, inplace=True)
sub = pd.merge(sub, team_loss_FGM, how='left', left_on=['Season', 'Team1'], right_on=['Season', 'LTeamID'])
sub = pd.merge(sub, team_win_FGM, how='left', left_on=['Season', 'Team2'], right_on=['Season', 'WTeamID'])
sub.drop(['LTeamID', 'WTeamID'], axis=1, inplace=True)
sub = pd.merge(sub, team_win_score, how='left', left_on=['Season', 'Team1'], right_on=['Season', 'WTeamID'])
sub = pd.merge(sub, team_loss_score, how='left', left_on=['Season', 'Team2'], right_on=['Season', 'LTeamID'])
sub = pd.merge(sub, team_loss_score, how='left', left_on=['Season', 'Team1'], right_on=['Season', 'LTeamID'])
sub = pd.merge(sub, team_win_score, how='left', left_on=['Season', 'Team2'], right_on=['Season', 'WTeamID'])
sub.drop(['WTeamID_x','LTeamID_x','LTeamID_y','WTeamID_y'], axis=1, inplace=True)
sub.head()

In [ ]:
sub['seed_diff'] = sub['Seed_x'] - sub['Seed_y']
sub.head()

In [ ]:
#Prepare test data
# sub = pd.read_csv('../input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv')
# sub.head()
# test = sub.copy()
# sub['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
# sub['Team1'] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
# sub['Team2'] = sub['ID'].apply(lambda x: int(x.split('_')[2]))
# sub = pd.merge(sub, data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'Team1'], right_on=['Season', 'TeamID'])
# sub = pd.merge(sub, data_dict['MNCAATourneySeeds'], how='left', left_on=['Season', 'Team2'], right_on=['Season', 'TeamID'])
# sub = pd.merge(sub, team_win_score, how='left', left_on=['Season', 'Team1'], right_on=['Season', 'WTeamID'])
# sub = pd.merge(sub, team_loss_score, how='left', left_on=['Season', 'Team2'], right_on=['Season', 'LTeamID'])
# sub = pd.merge(sub, team_loss_score, how='left', left_on=['Season', 'Team1'], right_on=['Season', 'LTeamID'])
# sub = pd.merge(sub, team_win_score, how='left', left_on=['Season', 'Team2'], right_on=['Season', 'WTeamID'])
# sub['seed_diff'] = sub['Seed_x'] - sub['Seed_y']
# sub.head()


In [ ]:
team_counts = data_dict['MTeamSpellings'].groupby('TeamID')['TeamNameSpelling'].count().reset_index()
team_counts.columns = ['TeamID', 'TeamSpellingCount']

In [ ]:
new_df = pd.merge(new_df, team_counts, how='left', left_on='Team1', right_on='TeamID')
new_df = new_df.drop(['TeamID'], axis=1)
new_df = pd.merge(new_df, team_counts, how='left', left_on='Team2', right_on='TeamID')
new_df = new_df.drop(['TeamID'], axis=1)

sub = pd.merge(sub, team_counts, how='left', left_on='Team1', right_on='TeamID')
sub = sub.drop(['TeamID'], axis=1)
sub = pd.merge(sub, team_counts, how='left', left_on='Team2', right_on='TeamID')
sub = sub.drop(['TeamID'], axis=1)

In [ ]:
new_df.head()

In [ ]:
sub.head()

In [ ]:
new_df = new_df.drop(['Season', 'Team1', 'Team2'], axis=1)
sub = sub.drop(['Pred', 'Season', 'Team1', 'Team2'], axis=1)
sub.columns = ['ID', 'Seed_1', 'Seed_2','WFGM_1','LFGM_1','LFGM_2','WFGM_2', 'WScore_1', 'LScore_1', 'LScore_2', 'WScore_2', 'seed_diff', 'TeamSpellingCount_x', 'TeamSpellingCount_y']
sub = sub[['ID', 'Seed_1', 'Seed_2', 'seed_diff', 'WFGM_1','LFGM_1','LFGM_2','WFGM_2','WScore_1', 'LScore_1', 'LScore_2', 'WScore_2', 'TeamSpellingCount_x', 'TeamSpellingCount_y']]
new_df = new_df.fillna(0)
sub = sub.fillna(0)

In [ ]:
X = new_df.drop(['target'], axis=1)
y = new_df['target']
X_test = sub.drop(['ID'], axis=1)

In [ ]:
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)

In [ ]:
def train_model(X, X_test, y, params, folds, model_type='lgb', plot_feature_importance=False, averaging='usual', model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    y_pred_valid_global = []
    y_valid_global = []
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.values[train_index], X.values[valid_index]
        y_train, y_valid = y.values[train_index], y.values[valid_index]
        
        if model_type == 'lgb':
            train_data = lgb.Dataset(X_train, label=y_train)
            valid_data = lgb.Dataset(X_valid, label=y_valid)
            
            model = lgb.train(params,
                    train_data,
                    num_boost_round=20000,
                    valid_sets = [train_data, valid_data],
                    verbose_eval=1000,
                    early_stopping_rounds = 200)
            
            y_pred_valid = model.predict(X_valid)
            score = roc_auc_score(y_valid, y_pred_valid)
            y_pred_valid_global = y_pred_valid
            y_valid_global = y_valid
            print(f'Fold {fold_n}. AUC: {score:.4f}.')
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X_train.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X_train.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X_train.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X_train.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid)[:, 1].reshape(-1,)
            score = roc_auc_score(y_valid, y_pred_valid)
            y_pred_valid_global = y_pred_valid
            y_valid_global = y_valid
            print(f'Fold {fold_n}. AUC: {score:.4f}.')
            # print('')
            
            y_pred = model.predict_proba(X_test)[:, 1]
        if model_type == 'sklearn1':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict(X_valid)
            score = roc_auc_score(y_valid, y_pred_valid)
            y_pred_valid_global = y_pred_valid
            y_valid_global = y_valid
            print(f'Fold {fold_n}. AUC: {score:.4f}.')
            # print('')
            
            y_pred = model.predict(X_test)
            
        if model_type == 'glm':
            model = sm.GLM(y_train, X_train, family=sm.families.Binomial())
            model_results = model.fit()
            model_results.predict(X_test)
            y_pred_valid = model_results.predict(X_valid).reshape(-1,)
            score = roc_auc_score(y_valid, y_pred_valid)
            
            y_pred = model_results.predict(X_test)
            
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=20000, learning_rate=0.1, loss_function='Logloss',  eval_metric='AUC')
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict_proba(X_valid)[:, 1].reshape(-1,)
            #y_pred_valid = model.predict_proba(X_valid)
            y_pred = model.predict_proba(X_test)[:, 1]
            y_pred_valid_global = y_pred_valid
            y_valid_global = y_valid
            score = roc_auc_score(y_valid, y_pred_valid)
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(log_loss(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values  
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importance()
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance, y_pred_valid_global, y_valid_global
        return oof, prediction, scores, y_pred_valid_global, y_valid_global
    
    else:
        return oof, prediction, scores, y_pred_valid_global, y_valid_global

In [ ]:
X1 = X.copy()
X_test1 = X_test.copy()

In [ ]:
scaler = StandardScaler()
X1[['WFGM_1','LFGM_1','LFGM_2','WFGM_2','WScore_1', 'LScore_1', 'LScore_2', 'WScore_2']] = scaler.fit_transform(X1[['WFGM_1','LFGM_1','LFGM_2','WFGM_2','WScore_1', 'LScore_1', 'LScore_2', 'WScore_2']])
X_test1[['WFGM_1','LFGM_1','LFGM_2','WFGM_2','WScore_1', 'LScore_1', 'LScore_2', 'WScore_2']] = scaler.transform(X_test1[['WFGM_1','LFGM_1','LFGM_2','WFGM_2','WScore_1', 'LScore_1', 'LScore_2', 'WScore_2']])

In [ ]:
from matplotlib import pyplot
model = linear_model.LogisticRegression(C=0.0001)
oof_lr, prediction_logistic_reg, scores, y_pred_valid_global, y_valid_global = train_model(X1, X_test1, y, params=None, folds=folds, model_type='sklearn', model=model)

test['Pred'] = prediction_logistic_reg
test.to_csv('logistic_reg_new.csv', index=False)
# print(y_pred_valid_global)
# print(y_valid_global)
# print(min(y_pred_valid_global))
log_reg_fpr, log_reg_ns_tpr, _ = roc_curve(y_valid_global, y_pred_valid_global)


# show the legend


In [ ]:

oof_lr, prediction_cat_reg, scores, y_pred_valid_global, y_valid_global = train_model(X1, X_test1, y, params=None, folds=folds, model_type='cat')

test['Pred'] = prediction_cat_reg
test.to_csv('cat_new.csv', index=False)
cat_reg_fpr, cat_reg_ns_tpr, _ = roc_curve(y_valid_global, y_pred_valid_global)


In [ ]:
model = LinearRegression()
oof_lr, prediction_linear_r, scores, y_pred_valid_global, y_valid_global = train_model(X1, X_test1, y, params=None, folds=folds, model_type='sklearn1', model=model)
test['Pred'] = prediction_linear_r
test.to_csv('linear_reg.csv', index=False)
lin_reg_fpr, lin_reg_ns_tpr, _ = roc_curve(y_valid_global, y_pred_valid_global)


In [ ]:
model = AdaBoostRegressor( n_estimators=100,learning_rate=0.005, loss='exponential')
oof_lr, prediction_ada, scores,y_pred_valid_global, y_valid_global = train_model(X1, X_test1, y, params=None, folds=folds, model_type='sklearn1', model=model)
test['Pred'] = prediction_ada
test.to_csv('adaboost.csv', index=False)
ada_reg_fpr, ada_reg_ns_tpr, _ = roc_curve(y_valid_global, y_pred_valid_global)


In [ ]:
model = KNeighborsRegressor(n_neighbors=20)
oof_lr, prediction_knn, scores, y_pred_valid_global, y_valid_global = train_model(X1, X_test1, y, params=None, folds=folds, model_type='sklearn1', model=model)
test['Pred'] = prediction_knn
test.to_csv('knn.csv', index=False)
knn_reg_fpr, knn_reg_ns_tpr, _ = roc_curve(y_valid_global, y_pred_valid_global)


In [ ]:
# from keras.layers import BatchNormalization
# def baseline_model():
#     # create model
#     model = Sequential()
#     model.add(Dense(20, input_dim=9, kernel_initializer='normal', activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dense(10, kernel_initializer='normal', activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.5))
# #model.add(BatchNormalization())
#     model.add(Dense(1, kernel_initializer='normal'))
#     # Compile model
#     model.compile(loss='mean_squared_logarithmic_error', optimizer='adam')
#     return model
# model = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=20, verbose=0)
# oof_lr, prediction_lr, scores = train_model(X1, X_test1, y, params=None, folds=folds, model_type='sklearn1', model=model)

In [ ]:
params = {'num_leaves': 8,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 5,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4}
oof_lgb, prediction_lgb, scores, y_pred_valid_global, y_valid_global = train_model(X, X_test, y, params=params, folds=folds, model_type='lgb', plot_feature_importance=True)
lgb_reg_fpr, lgb_reg_ns_tpr, _ = roc_curve(y_valid_global, y_pred_valid_global)


In [ ]:
pyplot.plot(lgb_reg_fpr, lgb_reg_ns_tpr,'go', label='Light GBD',color='green')
pyplot.plot(knn_reg_fpr, knn_reg_ns_tpr, 'g^', label='KNN',color='b')
pyplot.plot(log_reg_fpr, log_reg_ns_tpr, 'g<', label='Logistic', color='r')
pyplot.plot(ada_reg_fpr, ada_reg_ns_tpr, 'g>', label='Adaboost', color='c')
pyplot.plot(lin_reg_fpr, lin_reg_ns_tpr, 'gv', label='LinearRegressor', color='m')
pyplot.plot(cat_reg_fpr, cat_reg_ns_tpr, 'g*', label='Catboost', color='y')
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()
pyplot.show()

In [ ]:


test['Pred'] = prediction_lgb
test.to_csv('lgb.csv', index=False)

